In [ ]:
import wmpaws
import pandas as pd
pd.set_option('display.max_rows', None)

In [3]:
query = """
WITH genderEdits AS (
    SELECT  
        up_value AS gender,
        COUNT(r.rev_id) AS revision_count,
        YEAR(u.user_registration) AS registration_year
    FROM 
        user AS u
    LEFT JOIN 
        user_properties AS up ON u.user_id = up.up_user  
    LEFT JOIN
        revision AS r ON u.user_id = r.rev_id
    WHERE 
        YEAR(u.user_registration) BETWEEN 2019 AND 2024
    GROUP BY 
        up_value, YEAR(u.user_registration)
)
SELECT
    registration_year,
    SUM(CASE WHEN gender = 'male' THEN revision_count ELSE 0 END) AS Edits_made_by_male_editors,
    SUM(CASE WHEN gender = 'female' THEN revision_count ELSE 0 END) AS Edits_made_by_Female_editors,
    ROUND(
        SUM(CASE WHEN gender = 'male' THEN revision_count ELSE 0 END) / 
        NULLIF(SUM(CASE WHEN gender = 'female' THEN revision_count ELSE 0 END), 0), 2
    ) AS male_female_ratio
FROM 
    genderEdits
GROUP BY
    registration_year;
"""

In [4]:
wikis = ['pawiki', 'tewiki', 'hiwiki']

In [5]:
gender_counts_list = []

# loop through the wikis
for wiki in wikis:
    gender_counts_wiki = wmpaws.run_sql(query, wiki)
    gender_counts_wiki['wiki'] = wiki

    # append each wiki to a list
    gender_counts_list.append(gender_counts_wiki)

# combine all dataframe into one
gender_counts_all = pd.concat(gender_counts_list)

In [6]:
gender_counts_all.set_index(['wiki', 'registration_year', 'male_edits', 'female_edits', 'male_female_ratio'], verify_integrity=True)

Empty DataFrame
Columns: []
Index: [(pawiki, 2019, 21.0, 3.0, 7.0), (pawiki, 2020, 25.0, 1.0, 25.0), (pawiki, 2021, 20.0, 1.0, 20.0), (pawiki, 2022, 4.0, 1.0, 4.0), (pawiki, 2023, 4.0, 0.0, nan), (pawiki, 2024, 1.0, 0.0, nan), (tewiki, 2019, 44.0, 15.0, 2.93), (tewiki, 2020, 37.0, 5.0, 7.4), (tewiki, 2021, 26.0, 2.0, 13.0), (tewiki, 2022, 10.0, 3.0, 3.33), (tewiki, 2023, 4.0, 2.0, 2.0), (tewiki, 2024, 1.0, 2.0, 0.5), (hiwiki, 2019, 283.0, 38.0, 7.45), (hiwiki, 2020, 243.0, 34.0, 7.15), (hiwiki, 2021, 99.0, 11.0, 9.0), (hiwiki, 2022, 42.0, 17.0, 2.47), (hiwiki, 2023, 42.0, 5.0, 8.4), (hiwiki, 2024, 23.0, 2.0, 11.5)]